Exploração dos dados

In [1]:
import pandas as pd
import numpy as np

# Carregando o conjunto de dados
data = pd.read_csv('./matches_brasileirao_serie_a_2022.csv')

# verficando a quantidade de instâncias
instances = data.shape[0]
print('Número de instâncias: ' + str(instances))

# verificando a quantidade de atributos
attributes = data.shape[1]
print('Número de atributos: ' + str(attributes))

# Verificando a presença de dados ausentes
print(data.isna().sum())


Número de instâncias: 380
Número de atributos: 26
match_id                    0
stage                       0
date                        0
team_name_home              0
team_name_away              0
team_home_score             0
team_away_score             0
possession_home             0
possession_away             0
total_shots_home            0
total_shots_away            0
shots_on_target_home        0
shots_on_target_away        0
duels_won_home              0
duels_won_away              0
prediction_team_home_win    0
prediction_draw             0
prediction_team_away_win    0
prediction_quantity         0
location                    0
lineup_home                 0
lineup_away                 0
player_names_home           0
player_numbers_home         0
player_names_away           0
player_numbers_away         0
dtype: int64


Criando variáveis

In [2]:
# Separando atributos desejáveis
parsed_data = data.drop(
  labels=['team_name_home', 'team_name_away', 'stage', 'match_id', 'date', 'location', 
          'lineup_home', 'lineup_away', 'player_names_home', 'player_numbers_home', 
          'player_names_away', 'player_numbers_away', 'prediction_team_home_win', 
          'prediction_draw', 'prediction_team_away_win', 'prediction_quantity'],
  axis=1)

# Criando atributo necessário para classificação
parsed_data['both_scores_nonzero'] = np.where((parsed_data['team_home_score'] != 0) & 
                                              parsed_data['team_away_score']!=0, 1, 0)

parsed_data = parsed_data.drop(
  labels=['team_home_score', 'team_away_score'],
  axis=1)

parsed_data

,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,both_scores_nonzero
0,0.60,0.40,36,4,9,3,0.48,0.52,1
1,0.41,0.59,13,8,4,4,0.54,0.46,0
2,0.48,0.52,15,11,8,2,0.58,0.42,0
3,0.35,0.65,6,13,2,7,0.62,0.38,0
4,0.57,0.43,17,4,8,2,0.51,0.49,1
...,...,...,...,...,...,...,...,...,...
375,0.52,0.48,15,6,5,1,0.51,0.49,0
376,0.52,0.48,17,7,9,0,0.48,0.52,0
377,0.56,0.44,17,19,4,9,0.42,0.59,1
378,0.44,0.56,10,12,3,3,0.52,0.48,1


Verificando Outliers

In [3]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=parsed_data.columns, handle_unknown='return_nan', return_df=True, use_cat_names=True)
data_encoded = encoder.fit_transform(parsed_data)
data_encoded

from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor()
y_pred = lof.fit_predict(data_encoded)

from numpy import where

lofs_index = where(y_pred==-1)
lofs_index # -> Não há Outliers


ModuleNotFoundError: No module named 'category_encoders'

Dividindo os dados em um conjunto de treinamento e testes (80% por 20%)

In [ ]:
from sklearn.model_selection import train_test_split

# Divisão dos dados em conjunto de treinamento e teste
train_data, test_data = train_test_split(parsed_data, test_size=0.2, random_state=42)

# Verificando o tamanho dos conjuntos de treinamento e teste
print("Tamanho do conjunto de treinamento:", len(train_data))
print("Tamanho do conjunto de teste:", len(test_data))

Definindo o algotimo de treinamento

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Definindo as variáveis de entrada (features) e o alvo (target)
features = ['possession_home','possession_away','total_shots_home','total_shots_away',
            'shots_on_target_home','shots_on_target_away','duels_won_home','duels_won_away']
target = 'both_scores_nonzero'

# Separando as features e o alvo do conjunto de treinamento e teste
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

####################################

# Criando e treinando o modelo Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calculando a acurácia do modelo
accuracy_dt = accuracy_score(y_test, y_pred)
print("Acurácia do modelo Decision Tree:", accuracy_dt)

################################

from sklearn.ensemble import RandomForestClassifier

# Criando e treinando o modelo Random Forest
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_rf = model_rf.predict(X_test)

# Calculando a acurácia do modelo Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Acurácia do modelo Random Forest:", accuracy_rf)

################################

from sklearn.linear_model import LogisticRegression

# Criando e treinando o modelo Logistic Regression
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_lr = model_lr.predict(X_test)

# Calculando a acurácia do modelo Logistic Regression
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Acurácia do modelo Logistic Regression:", accuracy_lr)

######################################

from sklearn.svm import SVC

# Criando e treinando o modelo SVM
model_svm = SVC()
model_svm.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_svm = model_svm.predict(X_test)

# Calculando a acurácia do modelo SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Acurácia do modelo SVM:", accuracy_svm)

##########################################

from sklearn.naive_bayes import GaussianNB

# Criando e treinando o modelo Naive Bayes
model_nb = GaussianNB()
model_nb.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_nb = model_nb.predict(X_test)

# Calculando a acurácia do modelo Naive Bayes
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("Acurácia do modelo Naive Bayes:", accuracy_nb)

##############################

from sklearn.ensemble import GradientBoostingClassifier

# Criando e treinando o modelo Gradient Boosting
model_gb = GradientBoostingClassifier()
model_gb.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_gb = model_gb.predict(X_test)

# Calculando a acurácia do modelo Gradient Boosting
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Acurácia do modelo Gradient Boosting:", accuracy_gb)



In [ ]:
# Gerando gráfico de acurácias

import matplotlib.pyplot as plt
models = ['Decision Three', 'Random Forest', 'Logistic Regression', 'SVM', 'Naive Bayes', 
          'Gradient Boosting']
accuracys = [accuracy_dt, accuracy_rf, accuracy_lr, accuracy_svm, accuracy_nb, accuracy_gb]

plt.figure(figsize=(16, 4))

plt.bar(models, accuracys)

plt.xlabel('Modelos')
plt.ylabel('Acurácias')
plt.title('Gráfico de acurácias')

plt.show()

Validando o modelo

In [ ]:
from sklearn.model_selection import cross_val_score

# Aplicando validação cruzada Decision Tree
scores_dt = cross_val_score(model, X_train, y_train, cv=5)
print("DecisionTree")
print("Acurácia média:", scores_dt.mean())
print("Desvio padrão:", scores_dt.std())
print("\n")
#####################

# Aplicando validação cruzada Random Forest
scores_rf = cross_val_score(model_rf, X_train, y_train, cv=5)
print("RandomForest")
print("Acurácia média:", scores_rf.mean())
print("Desvio padrão:", scores_rf.std())
print("\n")

#####################

# Aplicando validação cruzada Logistic Regression
scores_lr = cross_val_score(model_lr, X_train, y_train, cv=5)
print("LogisticRegression")
print("Acurácia média:", scores_lr.mean())
print("Desvio padrão:", scores_lr.std())
print("\n")

#####################

# Aplicando validação cruzada SVM
scores_svm = cross_val_score(model_svm, X_train, y_train, cv=5)
print("SVM")
print("Acurácia média:", scores_svm.mean())
print("Desvio padrão:", scores_svm.std())
print("\n")

#####################

# Aplicando validação cruzada Naive Bayes
scores_nb = cross_val_score(model_nb, X_train, y_train, cv=5)
print("NaiveBayes")
print("Acurácia média:", scores_nb.mean())
print("Desvio padrão:", scores_nb.std())
print("\n")

#####################

# Aplicando validação cruzada Naive Bayes
scores_gb = cross_val_score(model_gb, X_train, y_train, cv=5)
print("GradientBoosting")
print("Acurácia média:", scores_gb.mean())
print("Desvio padrão:", scores_gb.std())
print("\n")

#####################

# Gerando gráfico da validação cruzada
models = ['Decision Three', 'Random Forest', 'Logistic Regression', 'SVM', 
          'Naive Bayes', ' Gradient Boosting']
means = [scores_dt.mean(), scores_rf.mean(), scores_lr.mean(), scores_svm.mean(), 
         scores_nb.mean(), scores_gb.mean()]

plt.figure(figsize=(16, 4))

width_bar = 0.35  # Largura das barras

# Posições das barras no eixo x
mean_bar_position = np.arange(len(models))
std_bar_position = mean_bar_position + width_bar

plt.bar(models, means, width_bar)

plt.xlabel('Modelos')
plt.ylabel('Média')
plt.title('Gráfico de validação cruzada')

plt.show()

stds = [scores_dt.std(), scores_rf.std(), scores_lr.std(), scores_svm.std(), scores_nb.std(), 
        scores_gb.std()]

plt.figure(figsize=(16, 4))
plt.bar(models, stds, width_bar)
plt.xlabel('Modelos')
plt.ylabel('Desvio padrão')
plt.title('Gráfico de validação cruzada')

plt.show()


In [ ]:
# Dados de teste
new_data = test_data 

# Realizando as previsões
new_features = new_data[features]
predictions = model.predict(new_features)

new_data['prediction_result'] = predictions
new_data['correct_prediction'] = (np.where((new_data['prediction_result'] == 
                                            new_data['both_scores_nonzero']), 1, 0))

correct_prediction_count = new_data['correct_prediction'].cumsum()
new_data['correct_prediction_count'] = correct_prediction_count

final_correct_count = new_data['correct_prediction_count'].iloc[-1]

print("Tamanho do conjunto de teste:", len(new_data))
print("Previsões Corretas:", final_correct_count)

In [ ]:
# Utilizando o modelo treinado para validar a base de jogos da serie B
data2 = pd.read_csv('./matches_brasileirao_serie_b_2022.csv')
parsed_data2 = data2.drop(
  labels=['team_name_home', 'team_name_away', 'stage', 'match_id', 'date', 'location', 
          'lineup_home', 'lineup_away', 'player_names_home', 'player_numbers_home', 
          'player_names_away', 'player_numbers_away', 'prediction_team_home_win', 
          'prediction_draw', 'prediction_team_away_win', 'prediction_quantity'],
  axis=1)

parsed_data2['both_scores_nonzero'] = np.where((parsed_data2['team_home_score'] != 0) & 
                                              (parsed_data2['team_away_score']!=0), 1, 0)

parsed_data2 = parsed_data2.drop(
  labels=['team_home_score', 'team_away_score'],
  axis=1)

# Verficando a correlação dos atributos
import seaborn as sns
sns.heatmap(data=parsed_data2.corr(method='spearman'), annot=True, cmap='YlGnBu', 
            fmt='.2f', square=True)

# Verificando a correlação dos atributos em relação a classe 'both_scores_nonzero'  
correlations = parsed_data2.corr(method='spearman')['both_scores_nonzero']
correlations_sorted = correlations.sort_values(ascending=False)
#print('Correlação em relação a "both_scores_nonzero"')
#print(correlations_sorted)

In [ ]:

# Fazendo previsões no conjunto de teste
predictions2 = model.predict(parsed_data2[features])

# Calculando a acurácia do modelo
accuracy = accuracy_score(parsed_data2[target], predictions2)
print("Acurácia do modelo Decision Tree:", accuracy)

In [ ]:
# Exibindo as previsões
print("Previsões:")

parsed_data2['prediction_result'] = predictions2
parsed_data2['correct_prediction'] = np.where((parsed_data2['prediction_result'] == parsed_data2['both_scores_nonzero']), 1, 0)


correct_prediction_count = parsed_data2['correct_prediction'].cumsum()
parsed_data2['correct_prediction_count'] = correct_prediction_count

final_correct_count = parsed_data2['correct_prediction_count'].iloc[-1]

print("Tamanho do conjunto de teste:", len(parsed_data2))
print("Previsões Corretas:", final_correct_count)